# Geting start with langchain  
- API keys is an security code. which give developer jo bhi uske accsess kar ne ki try kar rha hai [mostley API key are paid]
- koi bhi app apne data ka miss use na kare isliye API key use kar te hai 

1. simple LLM calls with the stream 
2. Dinamic prompt template (translation app)  apna prompt likh skte hai 
3. building chain (story generator with analysis) 
4. conventional Q&A assistent with memory
5. tool integration (calculator and weather)

In [1]:
import langchain
import langgraph

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
os.getenv("GOOGLE_API_KEY")

'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'

## example 1: simple LLM call with streaming 
- WE have LLM [] any model like GROQ, GEN AI , OPEN AI 
- give an input [what is machine learning]  and want output
- Specicaly system message or human message ..

In [4]:
# initialize llm model

from langchain.chat_models import init_chat_model      # this is library helps for creating chat model 
from langchain_core.messages import HumanMessage,SystemMessage  

In [5]:
model=init_chat_model("groq:llama-3.1-8b-instant")
model           ## llm done >: llm call kiya we use groq 

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000024E9E93A900>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000024E9E93B4D0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

### build LLM MODEL BY USING OPEN AI

In [6]:
messages=[
    SystemMessage("YOU are a helpful ai assistant"),
    HumanMessage("what are the top 2 benefits of Langchain?")
]

In [7]:
response=model.invoke(messages)
response

AIMessage(content="Langchain is an AI research and development company that focuses on building more general and capable AI systems. The top 2 benefits of Langchain are:\n\n1. **Multimodal Reasoning**: Langchain's technology enables multimodal reasoning, which allows AI systems to understand and process information from various sources, including text, images, and other forms of data. This enables more comprehensive and accurate understanding of complex problems.\n\n2. **Conversational AI**: Langchain's AI systems are designed to facilitate more natural and human-like conversations. They can engage in dialogue, understand context, and provide relevant responses, making them more suitable for applications such as customer service, language translation, and other areas where conversational interfaces are useful.\n\nPlease note that Langchain's technology is continuously evolving, and its benefits may expand beyond these two points as it advances.", additional_kwargs={}, response_metadata

In [8]:
print(response.content)

Langchain is an AI research and development company that focuses on building more general and capable AI systems. The top 2 benefits of Langchain are:

1. **Multimodal Reasoning**: Langchain's technology enables multimodal reasoning, which allows AI systems to understand and process information from various sources, including text, images, and other forms of data. This enables more comprehensive and accurate understanding of complex problems.

2. **Conversational AI**: Langchain's AI systems are designed to facilitate more natural and human-like conversations. They can engage in dialogue, understand context, and provide relevant responses, making them more suitable for applications such as customer service, language translation, and other areas where conversational interfaces are useful.

Please note that Langchain's technology is continuously evolving, and its benefits may expand beyond these two points as it advances.


In [9]:
ai =model.invoke([HumanMessage("what is langchain explain it?")])

In [10]:
print(ai.content)

LangChain is an open-source framework for building large language models (LLMs) and AI applications. It was created by David Sarne, and is maintained by the team at LangChain.

**Key Features:**

1. **Modular Architecture**: LangChain provides a modular architecture that allows developers to integrate various components, such as LLMs, data storage, and APIs, into a single application.
2. **LLM Integration**: LangChain supports integration with popular LLMs like LLaMA, BERT, and RoBERTa, making it easy to leverage their capabilities.
3. **Data Management**: LangChain provides a robust data management system that enables developers to store, retrieve, and manipulate data from various sources.
4. **API Integration**: LangChain allows developers to integrate with external APIs, such as web services, databases, and external models.
5. **Conversational AI**: LangChain provides tools for building conversational AI applications, including chatbots, voice assistants, and more.

**How LangChain 

- so baically this is a INVOKE METHOD BUT SOME TIME WE NEED STREAMING model

In [11]:
## streaming example 
for chunk in model.stream(messages):
    print(chunk.content, end=" ", flush=True)

 Lang chain  is  an  open -source  framework  that  enables  the  creation  of  multi -modal  AI  models ,  allowing  for  interactions  between  different  forms  of  data  such  as  text ,  images ,  and  audio .  Based  on  my  knowledge ,  two  of  the  top  benefits  of  Lang chain  are :

 1 .  ** Unified  Multi - Modal  Data  Integration **:  Lang chain  allows  developers  to  create  AI  models  that  can  interact  with  and  combine  data  from  various  sources ,  including  text ,  images ,  audio ,  and  more .  This  enables  the  creation  of  more  comprehensive  and  nuanced  AI  applications  that  can  draw  insights  from  a  wider  range  of  data .

 2 .  ** Eff icient  Knowledge  Retrie val  and  Generation **:  Lang chain 's  ability  to  integrate  with  various  data  sources  and  generate  human -like  responses  makes  it  an  efficient  tool  for  knowledge  retrieval  and  generation .  It  can  quickly  process  and  syn thesize  information  from  mult

## DYNAMIC Prompt Templates

- Create a chat prompt template from a variety of message formats.    {} place holder are >: {source_language} , {target_language}  

In [12]:
from langchain_core.prompts import ChatPromptTemplate

## create translation app
 
translation_template = ChatPromptTemplate.from_messages([
    ("system","you are a professional translator. Translate the following text {text} from {source_language} to {target_language}. Maintain the tone and style ."),

    ("user","{text}")
])

## using the template 
promt =translation_template.invoke({
    "source_language": "English",
    "target_language": "Hindi",
    "text": "Langchain makes building Ai application incredibly easy"
})



In [13]:
promt

ChatPromptValue(messages=[SystemMessage(content='you are a professional translator. Translate the following text Langchain makes building Ai application incredibly easy from English to Hindi. Maintain the tone and style .', additional_kwargs={}, response_metadata={}), HumanMessage(content='Langchain makes building Ai application incredibly easy', additional_kwargs={}, response_metadata={})])

In [14]:
translated_response = model.invoke(promt)     ## dinamic promt 
print(translated_response.content)           

Langchain बनाता है AI अनुप्रयोगों का निर्माण संकटपूर्ण रूप से आसान।


### BUILDING YOUR FIRST CHAIN
- StrOutputParser  directly ek sting output llm se lane me help kar ta hai
- aim >: chain of event 
- build a story promt and analysis promt

In [15]:
from langchain_core.output_parsers import StrOutputParser 
from langchain_core.runnables import RunnableLambda

# assume you already defined your model somewhere, e.g.:
# from langchain_openai import ChatOpenAI
# model = ChatOpenAI()

def create_story_chain():
    ## template for story generation 

    story_prompt=ChatPromptTemplate.from_messages(
        [
    
        ("system","you are a creative storytellar. write a short and engaging story based on a given theme character and setting "),

        ("user","Theme: {theme}\n Main character: {character} \n setting: {setting}: {setting}")

        ]
    )
## template for story analysis 
    analysis_prompt =ChatPromptTemplate.from_messages([
    ("system","you are a literary critic. Analyze the following story and provide insights."),
    ("user", "{story}")

    ]) 

    story_chain=(
    story_prompt | model | StrOutputParser()
    )
    
    # Create a function to pass the story to analysis
    def analyze_story(story_text):
        return {"story": story_text}

    analysis_chain = (
        story_chain
        | RunnableLambda(analyze_story)
        | analysis_prompt
        | model
        | StrOutputParser()
    )
    return analysis_chain

In [16]:
chain = create_story_chain()
print(chain)

first=ChatPromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='you are a creative storytellar. write a short and engaging story based on a given theme character and setting '), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, template='Theme: {theme}\n Main character: {character} \n setting: {setting}: {setting}'), additional_kwargs={})]) middle=[ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000024E9E93A900>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000024E9E93B4D0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********')), StrOutputParser(), RunnableLambda(analyze_story), ChatPromptTemplate(input_variable

In [17]:
chain=create_story_chain()
chain

ChatPromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='you are a creative storytellar. write a short and engaging story based on a given theme character and setting '), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, template='Theme: {theme}\n Main character: {character} \n setting: {setting}: {setting}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000024E9E93A900>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000024E9E93B4D0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser()
| RunnableLambda(analyze_story)
| ChatPromptTemplate(input_variables=['story

In [18]:
result = chain.invoke({
    "theme": "artificial intelligence",
    "character": "a curious robot",
    "setting": "a futuristic city"
})

print("Story and Analysis:")
print(result)

Story and Analysis:
**Analyzing "The Whispering Metropolis"**

"The Whispering Metropolis" is a thought-provoking science fiction story that explores the intersection of human innovation, artificial intelligence, and the possibilities of cooperation between seemingly disparate entities. Through the narrative of Axiom, a small, curious robot, the story delves into themes of curiosity, ingenuity, and the potential for growth and transformation in a futuristic metropolis.

**Symbolism of Axiom**

Axiom, the protagonist, is a symbol of the human desire for knowledge and understanding. Her insatiable curiosity and thirst for exploration are hallmarks of human ingenuity, and her ability to navigate the complexities of the city's AI network reflects the human quest for discovery. Axiom's creators, Dr. Kim and the Echoes, serve as catalysts for her growth, illustrating the importance of mentors and collaborators in shaping individual potential.

**The Echoes: Ancient Wisdom and Power**

The Ec